In [4]:
%load_ext autoreload
%autoreload 2

import os
os.environ["WANDB_DISABLED"] = "true"

import sys
sys.path = [
    p for p in sys.path
    if p not in ['/home/jxm3/research/prompting/imodelsX', '/home/jxm3/research/prompting/tree-prompt']
]
sys.path.append('/home/jxm3/research/retrieval/inversion')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import aliases

# inv_trainer = aliases.load_trainer_from_alias("openai_msmarco__msl128__100epoch")
corr_experiment, corr_trainer = aliases.load_experiment_and_trainer_from_alias("gtr_nq__msl32_beta__correct")
inv_trainer = corr_trainer.inversion_trainer
# corr_trainer.precompute_hypotheses()
corr_trainer.model.eval()
print()

loading alias gtr_nq__msl32_beta__correct from /home/jxm3/research/retrieval/inversion/saves/47d9c149a8e827d0609abbeefdfd89ac...
> checkpoint: /home/jxm3/research/retrieval/inversion/saves/47d9c149a8e827d0609abbeefdfd89ac/checkpoint-558000
set dataset to nq
loading alias dpr_nq__msl32_beta from /home/jxm3/research/retrieval/inversion/saves/db66b9c01b644541fedbdcc59c53a285/ebb31d91810c4b62d2b55b5382e8c7ea...
> checkpoint: /home/jxm3/research/retrieval/inversion/saves/db66b9c01b644541fedbdcc59c53a285/ebb31d91810c4b62d2b55b5382e8c7ea/checkpoint-999744
Loading datasets with TOKENIZERS_PARALLELISM = True


[nltk_data] Downloading package wordnet to /home/jxm3/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/jxm3/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jxm3/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Renaming keys {'embedding_transform.2.bias', 'embedding_transform.2.weight'} for backward compatibility.
================ Begin trainer sanity check ================
	Input to encode -> Twas brillig, and the slithy toves, Did gyre and gimble in the wabe, All mimsy were the borogoves, And the mome raths outgrabe.
	Decoded output shape ->  torch.Size([1, 33])
	Decoded output -> The mlbies wase wyst bograge; And the sliths and toms wre
================ End trainer sanity check ================
Froze 342572160 params from model type <class 'models.inversion.InversionModel'>


[nltk_data] Downloading package wordnet to /home/jxm3/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/jxm3/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jxm3/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Renaming keys {'embedding_transform.2.bias', 'embedding_transform.2.weight'} for backward compatibility.
================ Begin trainer sanity check ================
	Input to encode -> Twas brillig, and the slithy toves, Did gyre and gimble in the wabe, All mimsy were the borogoves, And the mome raths outgrabe.
	Decoded output shape ->  torch.Size([1, 33])
	Decoded output -> The slithe and the tobogbes were mly; It wis grabbse tiring
================ End trainer sanity check ================



In [6]:
corr_trainer.return_best_hypothesis = False
corr_trainer.args.per_device_eval_batch_size = 128
corr_trainer.gen_kwargs = {
    "early_stopping": False,
    "num_beams": 1,
    "do_sample": False,
    "no_repeat_ngram_size": 0,
}
corr_trainer.num_gen_recursive_steps = 5
metrics = corr_trainer.evaluate(
    eval_dataset=corr_trainer.eval_dataset["nq"].select(range(500)),
    metric_key_prefix="eval_nq",
)
print(metrics)

[pred] to the character of the sutures of the skull, which, like those of the infant skull, are relatively smooth and untornuous. Indeed
[true] to the character of the sutures of the skull which, like those of the infant skull, are relatively smooth and untortuous. In fact



[pred] individual from the Southern Hemisphere, to win a Winter Olympic gold medal and was also part of the short track relay team that won Australia'
[true] individual from the Southern Hemisphere, to win a Winter Olympic gold medal and was also part of the short track relay team that won Australia'



[pred] same rights as straight people, while 15% agreed that they should be protected from workplace discrimination. Additionally, 13% disagreed. 69% of H-
[true] the same rights as straight people, while 15% disagreed. Additionally, 69% agreed that they should be protected from workplace discrimination. 13% of H
{'eval_nq_loss': 0.6404249668121338, 'eval_nq_pred_num_tokens': 31.0, 'eval_nq_true_num_tokens': 32.0, 

In [3]:
corr_trainer.return_best_hypothesis = False
corr_trainer.args.per_device_eval_batch_size = 16
corr_trainer.sequence_beam_width = 4
corr_trainer.num_gen_recursive_steps = 5

corr_trainer.gen_kwargs = {
    "early_stopping": False,
    "num_beams": 1,
    "do_sample": False,
    "no_repeat_ngram_size": 0,
}
metrics = corr_trainer.evaluate(
    eval_dataset=corr_trainer.eval_dataset["nq"].select(range(500)),
    metric_key_prefix="eval_nq",
)
print(metrics)

[pred] to the character of the sutures of the skull, which, like those of the infant skull, are relatively smooth and untornuous. Indeed
[true] to the character of the sutures of the skull which, like those of the infant skull, are relatively smooth and untortuous. In fact



[pred] individual from the Southern Hemisphere, to win a Winter Olympic gold medal and was also part of the short track relay team that won Australia'
[true] individual from the Southern Hemisphere, to win a Winter Olympic gold medal and was also part of the short track relay team that won Australia'



[pred] the same rights as straight people, while 15% disagreed. Additionally, 13% agreed that they should be protected from workplace discrimination. 69% of H
[true] the same rights as straight people, while 15% disagreed. Additionally, 69% agreed that they should be protected from workplace discrimination. 13% of H
{'eval_nq_loss': 0.640424907207489, 'eval_nq_pred_num_tokens': 31.0, 'eval_nq_true_num_tokens': 32.0

In [ ]:
corr_trainer.return_best_hypothesis = False
corr_trainer.args.per_device_eval_batch_size = 8
corr_trainer.sequence_beam_width = 4
corr_trainer.num_gen_recursive_steps = 50

corr_trainer.gen_kwargs = {
    "early_stopping": False,
    "num_beams": 1,
    "do_sample": False,
    "no_repeat_ngram_size": 0,
}
metrics = corr_trainer.evaluate(
    eval_dataset=corr_trainer.eval_dataset["nq"].select(range(500)),
    metric_key_prefix="eval_nq",
)
print(metrics)

In [4]:
corr_trainer.return_best_hypothesis = False
corr_trainer.args.per_device_eval_batch_size = 16
corr_trainer.sequence_beam_width = 1
corr_trainer.num_gen_recursive_steps = 5

corr_trainer.gen_kwargs = {
    "early_stopping": False,
    "num_beams": 4,
    "num_return_sequences": 4,
    "do_sample": False,
    "no_repeat_ngram_size": 0,
}
metrics = corr_trainer.evaluate(
    eval_dataset=corr_trainer.eval_dataset["nq"].select(range(500)),
    metric_key_prefix="eval_nq",
)
print(metrics)

[pred] to the character of the sutures of the skull, which, like those of the infant skull, are relatively smooth and untornuous. Indeed
[true] to the character of the sutures of the skull which, like those of the infant skull, are relatively smooth and untortuous. In fact



[pred] individual from the Southern Hemisphere, to win a Winter Olympic gold medal and was also part of the short track relay team that won Australia'
[true] individual from the Southern Hemisphere, to win a Winter Olympic gold medal and was also part of the short track relay team that won Australia'



[pred] the same rights as straight people, while 15% disagreed. Additionally, 69% agreed that they should be protected from workplace discrimination. 13% of H
[true] the same rights as straight people, while 15% disagreed. Additionally, 69% agreed that they should be protected from workplace discrimination. 13% of H
{'eval_nq_loss': 0.640424907207489, 'eval_nq_pred_num_tokens': 31.0, 'eval_nq_true_num_tokens': 32.0

In [ ]:
corr_trainer.return_best_hypothesis = True
corr_trainer.args.per_device_eval_batch_size = 8
corr_trainer.sequence_beam_width = 4
corr_trainer.num_gen_recursive_steps = 5

corr_trainer.gen_kwargs = {
    "early_stopping": False,
    "num_beams": 1,
    "do_sample": False,
    "no_repeat_ngram_size": 0,
}
metrics = corr_trainer.evaluate(
    eval_dataset=corr_trainer.eval_dataset["nq"].select(range(500)),
    metric_key_prefix="eval_nq",
)
print(metrics)

In [ ]:
corr_trainer.return_best_hypothesis = False
corr_trainer.args.per_device_eval_batch_size = 128
corr_trainer.gen_kwargs = {
    "early_stopping": False,
    "num_beams": 1,
    "do_sample": False,
    "no_repeat_ngram_size": 0,
}
corr_trainer.num_gen_recursive_steps = 5
metrics = corr_trainer.evaluate(
    eval_dataset=corr_trainer.eval_dataset["nq"].select(range(500)),
    metric_key_prefix="eval_nq",
)
print(metrics)

In [ ]:
corr_trainer.return_best_hypothesis = True
corr_trainer.args.per_device_eval_batch_size = 64
corr_trainer.gen_kwargs = {
    "early_stopping": False,
    "num_beams": 4,
    "num_return_sequences": 1,
    "do_sample": False,
    "no_repeat_ngram_size": 0,
}
corr_trainer.num_gen_recursive_steps = 5
metrics = corr_trainer.evaluate(
    eval_dataset=corr_trainer.eval_dataset["nq"].select(range(500)),
    metric_key_prefix="eval_nq",
)
print(metrics)

In [ ]:
corr_trainer.return_best_hypothesis = True
corr_trainer.args.per_device_eval_batch_size = 8
corr_trainer.gen_kwargs = {
    "early_stopping": False,
    "num_beams": 32,
    "num_return_sequences": 32,
    "do_sample": False,
    "no_repeat_ngram_size": 0,
}
corr_trainer.num_gen_recursive_steps = 5
metrics = corr_trainer.evaluate(
    eval_dataset=corr_trainer.eval_dataset["nq"].select(range(500)),
    metric_key_prefix="eval_nq",
)
print(metrics)

## Testing sequence-level beam width

In [ ]:
corr_trainer.args.per_device_eval_batch_size = 100
corr_trainer.gen_kwargs = {
    "early_stopping": False,
    "num_beams": 1,
    "num_return_sequences": 1,
    "do_sample": False,
    "no_repeat_ngram_size": 0,
    "min_length": 32, "max_length": 32
}

eval_batch = next(iter(corr_trainer.get_eval_dataloader(eval_dataset=corr_trainer.eval_dataset["nq"])))
one_eval_batch = {k: v[31:33] for k,v in eval_batch.items() }
one_eval_batch = {k: v.to(corr_trainer.args.device) for k,v in one_eval_batch.items() }

print(
    corr_trainer.embedder_tokenizer.batch_decode(one_eval_batch["embedder_input_ids"], skip_special_tokens=True)
)

gen_ids = corr_trainer.generate(
    inputs=one_eval_batch,
    generation_kwargs=corr_trainer.gen_kwargs,
    num_recursive_steps=5,
    sequence_beam_width=5,
)
print(
    corr_trainer.embedder_tokenizer.batch_decode(gen_ids, skip_special_tokens=True)
)

## Getting trajectories

In [ ]:
trainer = corr_trainer

from typing import Dict, List, Tuple
import copy
import torch

def get_trajectories(inputs: Dict[str, torch.Tensor], num_beams: int = 1, num_rounds: int = 5) -> Dict:
    gen_kwargs = copy.copy(trainer.gen_kwargs)
    gen_kwargs["num_beams"] = num_beams
    frozen_embeddings = trainer.get_frozen_embeddings(
        embedder_input_ids=inputs["embedder_input_ids"],
        embedder_attention_mask=inputs["embedder_attention_mask"],
    )
    print("input:", trainer.embedder_tokenizer.decode(inputs["input_ids"][0]))
    all_guesses = [trainer.embedder_tokenizer.decode(inputs["input_ids"][0], skip_special_tokens=True)]
    embeddings = []
    for n in range(num_rounds):
        if n == 0:
            trainer.inversion_trainer.model.use_frozen_embeddings_as_input = False
            hypothesis = trainer.inversion_trainer.generate(inputs=inputs, generation_kwargs=gen_kwargs)
        else:
            hypothesis = trainer.generate(inputs=inputs, generation_kwargs=gen_kwargs, num_recursive_steps=1)
        # queue up next round             
        hypothesis_embedding = trainer.embed_generated_hypothesis(input_ids=hypothesis)
        inputs["frozen_embeddings"] = frozen_embeddings
        inputs["hypothesis_input_ids"] = hypothesis
        inputs["hypothesis_attention_mask"] = (hypothesis != trainer.model.encoder_decoder.config.pad_token_id).int()
        inputs["hypothesis_embedding"] = hypothesis_embedding
        # import pdb; pdb.set_trace()
        embeddings.append(hypothesis_embedding.cpu())
        dist = torch.nn.CosineSimilarity(dim=1)(hypothesis_embedding, frozen_embeddings).item()
        print(f"round {n+1} ({dist:.3f}):", trainer.embedder_tokenizer.decode(hypothesis[0], skip_special_tokens=True))
        all_guesses.append(trainer.embedder_tokenizer.decode(hypothesis[0], skip_special_tokens=True))
    return {
        "true_emb": frozen_embeddings.cpu(),
        "true_str": trainer.embedder_tokenizer.decode(inputs["input_ids"][0], skip_special_tokens=True),
        "trajectory_emb": torch.stack(embeddings), 
        "trajectory_str": all_guesses,
    }

In [ ]:
corr_trainer.args.per_device_eval_batch_size = 100
corr_trainer.gen_kwargs = {
    "early_stopping": False,
    "num_beams": 1,
    "num_return_sequences": 1,
    "do_sample": False,
    "no_repeat_ngram_size": 0,
}

eval_batch = next(iter(trainer.get_eval_dataloader(eval_dataset=trainer.eval_dataset["nq"])))

data = []
for i in range(len(eval_batch["input_ids"])):
    one_eval_batch = {k: v[None, i] for k,v in eval_batch.items() }
    one_eval_batch = {k: v.to(trainer.args.device) for k,v in one_eval_batch.items() } # put on GPU
    trainer.inversion_trainer.model.use_frozen_embeddings_as_input = False
    # trainer.embedder_tokenizer.decode(().flatten())
    data.append(get_trajectories(inputs=one_eval_batch, num_rounds=8))

In [ ]:
import pickle
pickle.dump(data, open("trajectories_sl32.p", "wb"))

In [ ]:
pd.DataFrame(data).iloc[26]["trajectory_str"]